# Python Package Choice

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline 
from textblob import TextBlob
import sklearn


# Load the dataset
df = pd.read_csv('amazon.csv')

/Users/Yoni/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/var/folders/3h/r5l1t4656pbc6bzdv94m9wrm0000gn/T/ipykernel_14514/2010257997.py:12: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('amazon.csv')


In [2]:
# Display the first few rows of the dataframe
df.head()

# Display the concise summary of the DataFrame
# print("DataFrame Information:")
df.info()

# Display the shape of the DataFrame
print("\nDataFrame Shape:")
print(df.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

## preprocessing

In [3]:
df_copy = df.copy()

# Dropping columns with predominantly missing values or not relevant
columns_to_drop = ['reviews.userCity', 'reviews.userProvince', 'reviews.didPurchase', 'reviews.id']
df_copy.drop(columns=columns_to_drop, inplace=True)

# Filling missing values in 'name' with "Unknown"
df_copy['name'].fillna('Unknown', inplace=True)

def clean_text(text):
    if not isinstance(text, str):
        return ""  # Return an empty string for non-string inputs
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df_copy['reviews.text'] = df_copy['reviews.text'].apply(clean_text)

df_copy['reviews.date'] = pd.to_datetime(df_copy['reviews.date'])

# Example: Filling missing ratings with the median rating
median_rating = df_copy['reviews.rating'].median()
df_copy['reviews.rating'].fillna(median_rating, inplace=True)
# df_copy

### TextBlob Test

In [4]:
text = "This product has been excellent in performance but I had issues with its customer service."
blob = TextBlob(text)
print(f"TextBlob Sentiment: {blob.sentiment}")


TextBlob Sentiment: Sentiment(polarity=1.0, subjectivity=1.0)


In [6]:
reviews = [
     "This product has been excellent in performance but I had issues with its customer service.",
    "Absolutely love this product, it works wonders!",
    "Terrible, did not work as advertised.",
    "Okay, but I've seen better performance from cheaper products.",
    "Amazing quality, exceeded my expectations.",
    "The worst customer service experience ever.",
    "Pretty good, but not worth the price.",
    "This product saved me so much time!",
    "Not happy with the purchase. It broke after a week.",
    "Excellent durability and performance. Highly recommend.",
    "Mediocre at best, I wouldn't buy it again."
]

for review in reviews:
    blob = TextBlob(review)
    print(f"Review: {review}\nTextBlob Sentiment: {blob.sentiment}\n")


Review: This product has been excellent in performance but I had issues with its customer service.
TextBlob Sentiment: Sentiment(polarity=1.0, subjectivity=1.0)

Review: Absolutely love this product, it works wonders!
TextBlob Sentiment: Sentiment(polarity=0.625, subjectivity=0.6)

Review: Terrible, did not work as advertised.
TextBlob Sentiment: Sentiment(polarity=-1.0, subjectivity=1.0)

Review: Okay, but I've seen better performance from cheaper products.
TextBlob Sentiment: Sentiment(polarity=0.5, subjectivity=0.5)

Review: Amazing quality, exceeded my expectations.
TextBlob Sentiment: Sentiment(polarity=0.6000000000000001, subjectivity=0.9)

Review: The worst customer service experience ever.
TextBlob Sentiment: Sentiment(polarity=-1.0, subjectivity=1.0)

Review: Pretty good, but not worth the price.
TextBlob Sentiment: Sentiment(polarity=0.26666666666666666, subjectivity=0.5666666666666668)

Review: This product saved me so much time!
TextBlob Sentiment: Sentiment(polarity=0.25, 

### Hugging Face Transformers Test

In [5]:
nlp = pipeline("sentiment-analysis")
text = "This product has been excellent in performance but I had issues with its customer service."
result = nlp(text)
print(f"Hugging Face Sentiment: {result}")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9973146319389343}]


In [7]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis")

reviews = [
    "This product has been excellent in performance but I had issues with its customer service.",
    "Absolutely love this product, it works wonders!",
    "Terrible, did not work as advertised.",
    "Okay, but I've seen better performance from cheaper products.",
    "Amazing quality, exceeded my expectations.",
    "The worst customer service experience ever.",
    "Pretty good, but not worth the price.",
    "This product saved me so much time!",
    "Not happy with the purchase. It broke after a week.",
    "Excellent durability and performance. Highly recommend.",
    "Mediocre at best, I wouldn't buy it again."
]

for review in reviews:
    result = nlp(review)
    print(f"Review: {review}\nHugging Face Sentiment: {result}\n")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Review: This product has been excellent in performance but I had issues with its customer service.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9973146319389343}]

Review: Absolutely love this product, it works wonders!
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998862743377686}]

Review: Terrible, did not work as advertised.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9994440674781799}]

Review: Okay, but I've seen better performance from cheaper products.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9877060055732727}]

Review: Amazing quality, exceeded my expectations.
Hugging Face Sentiment: [{'label': 'POSITIVE', 'score': 0.9998855590820312}]

Review: The worst customer service experience ever.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.999789297580719}]

Review: Pretty good, but not worth the price.
Hugging Face Sentiment: [{'label': 'NEGATIVE', 'score': 0.9121016263961792}]

Review: This product saved me so mu


### TextBlob
- **Pros:** Excellent for quick sentiment analysis and basic NLP tasks. Its simplicity and ease of use are ideal for straightforward text processing, like cleaning and sentiment assessment in your dataset.
- **Cons:** May not offer the depth required for more nuanced sentiment analysis or complex NLP tasks that could be beneficial in fully understanding customer feedback nuances.

### Scikit-learn
- **Pros:** With its comprehensive machine learning toolkit, it's great for classification, regression, and clustering, which can be applied to predict product popularity based on review ratings. Also useful for preprocessing and feature extraction.
- **Cons:** Lacks the direct NLP capabilities for sentiment analysis without additional libraries or custom implementations. It's more focused on numerical and categorical data processing.

### Transformers (Hugging Face)
- **Pros:** Provides access to state-of-the-art NLP models capable of deep sentiment analysis and understanding complex language patterns. Highly suitable for extracting sophisticated insights from customer reviews.
- **Cons:** Requires more computational resources and has a steeper learning curve. May introduce complexity in preprocessing steps due to the need for tokenization and input formatting specific to these advanced models.

Given the preprocessing code you're working with, focusing on cleaning text and handling missing data, each library offers distinct advantages:
- **TextBlob** is the most straightforward for directly applying sentiment analysis after your initial preprocessing.
- **Scikit-learn** is highly beneficial for further data manipulation, especially in preparing your dataset for machine learning tasks.
- **Transformers** offer the most advanced capabilities for sentiment analysis, assuming the data is preprocessed to fit the model requirements. 

